# Senegal Market Access
Using the global friction surface (Weiss et al. 2019) calculate market access to cities > 50,000 in Senegal and within 50km buffer of the country

In [ ]:
import sys
import os
import rasterio

import geopandas as gpd
import skimage.graph as graph

sys.path.insert(0, r"../../src")

import GOSTnetsraster.market_access as ma

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
iso3 = "SEN"
working_folder = "C:/WBG/Work/Projects/SEN_MA"
data_folder = os.path.join(working_folder, "data")
results_folder = os.path.join(working_folder, "results")

for cfolder in [working_folder, data_folder, results_folder]:
    if not os.path.exists(cfolder):
        os.makedirs(cfolder)

# define input data
sen_friction = os.path.join(data_folder, "SEN_friction_surface.tif")
aoi_file = os.path.join(data_folder, "SEN_AOI_buffer.geojson")
city_file = os.path.join(data_folder, "SEN_cities.geojson")

if not os.path.exists(aoi_file):
    global_admin = r"C:\WBG\Work\data\ADMIN\ADM0.shp"
    all_admin = gpd.read_file(global_admin)
    sen_admin = all_admin.loc[all_admin["ISO_A3"] == iso3]
    sen_admin = sen_admin.to_crs(3857)
    sen_admin["geometry"] = sen_admin.buffer(50000)  # 50km buffer
    sen_admin.to_file(aoi_file, driver="GeoJSON")
in_aoi = gpd.read_file(aoi_file)

if not os.path.exists(sen_friction):
    import GOSTrocks.rasterMisc as rMisc

    global_friction = (
        r"C:\WBG\Work\data\FRICTION\2020_motorized_friction_surface.geotiff"
    )
    cur_r = rMisc.clipRaster(
        rasterio.open(global_friction), in_aoi, sen_friction, crop=False
    )
in_friction = rasterio.open(sen_friction)

if not os.path.exists(city_file):
    global_cities = r"C:/WBG/Work/data/URBAN/GHS_UCDB_GLOBE_R2024A.gpkg"
    all_city = gpd.read_file(global_cities)
    in_aoi = in_aoi.to_crs(all_city.crs)
    sel_city = gpd.sjoin(all_city, in_aoi, how="inner")
    sel_city.loc[
        :,
        [
            "ID_UC_G0",
            "GC_UCN_MAI_2025",
            "GC_CNT_GAD_2025",
            "GC_POP_TOT_2025",
            "geometry",
        ],
    ].to_file(city_file, driver="GeoJSON")
dests = gpd.read_file(city_file)

In [3]:
frictionD = in_friction.read()[0, :, :]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [5]:
# Convert the cities geometry to centroid, then project to crs of friction surface
dests["geometry"] = dests["geometry"].centroid
dests = dests.to_crs(in_friction.crs)

# calculate travel time to nearest cities
travel_costs, traceback = ma.calculate_travel_time(in_friction, mcp, dests)
travel_costs = travel_costs.astype(in_friction.meta["dtype"])
with rasterio.open(
    os.path.join(results_folder, "least_cost_travel_time_cities.tif"),
    "w",
    **in_friction.meta,
) as out_file:
    out_file.write_band(1, travel_costs)

C:\Users\WB411133\AppData\Local\Temp\ipykernel_20612\3328050362.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dests['geometry'] = dests['geometry'].centroid


In [99]:
# calculate gravity to nearest cities, using the population column GC_POP_TOT_2025
gravity_col = "GC_POP_TOT_2025"
gravity_file = os.path.join(
    results_folder, f"gravity_cities_{gravity_col}_one_band.tif"
)
gravity = ma.calculate_gravity(
    in_friction, mcp, dests, gravity_col, outfile=gravity_file, decayVals=[0.0003850]
)

0:Dakar
1:Serrekunda
2:M'bour
3:Brikama
4:Thiès
5:Gabu
6:Basse Santa Su
7:Kaédi
8:Darou Khoudoss
9:Tivaouane
10:Bambey
11:Saint-Louis
12:Fatick
13:Ziguinchor
14:Bignona
15:Diourbel
16:Louga
17:Kaolack
18:Darou Mousty
19:Touba
20:Sédhiou
21:Richard-Toll
22:Kaffrine
23:Dahra Djoloff
24:Kolda
25:Koungheul
26:Vélingara
27:Médina Gounass
28:Tambacounda
29:Kedougou


In [92]:
dests[gravity_col].values

array([3970201.794  , 1313464.797  ,  506407.1053 ,  213625.807  ,
        483414.6727 ,   86490.69096,   64316.98188,   61350.09904,
         65892.27379,  138729.0962 ,   59471.7285 ,  263603.0204 ,
         64861.15138,  288803.6113 ,   82776.49342,  220539.7198 ,
        187360.2719 ,  406200.8382 ,   53486.55792, 1170258.55   ,
         60980.41171,   80805.62807,   94450.60387,   73946.19702,
        199859.8099 ,   63226.41215,   84758.86684,   67376.52666,
        241376.9138 ,   60486.29122])

In [93]:
row = 447
col = 153

In [94]:
gravity["costs"][row, col]

array([295.74580021,   2.5       , 251.93713479,  14.08291267,
       278.14208986, 472.22487016, 293.41313211, 675.38490775,
       318.49574595, 291.97014499, 237.46132868, 409.59369927,
       201.77604626, 147.84057723, 125.08470354, 218.47114054,
       360.15288015, 169.5391526 , 285.18195993, 252.92339744,
       226.51903381, 489.17990662, 218.59966616, 321.65045033,
       284.97088414, 288.52251575, 378.22748597, 419.37604488,
       388.28293236, 561.76316821])

In [95]:
gravity["gravity"][row, col]

9765624.055253685

In [96]:
row = 83
col = 285
gravity["costs"][row, col]

array([277.45820866, 510.43579912, 278.36976152, 520.99268432,
       242.92991609, 823.1494935 , 649.00421489, 321.97998791,
       245.67698715, 226.41620709, 273.53452968, 102.17645351,
       327.83287716, 654.75034888, 631.99447519, 292.80033028,
       153.51074083, 342.38950923, 260.03420821, 297.2155371 ,
       658.76768432,  21.70103989, 392.52224029, 261.87110183,
       683.16142648, 462.44508989, 644.76990017, 630.45422873,
       555.9513403 , 717.40152416])

In [97]:
gravity["gravity"][row, col]

9388545.490043154

In [98]:
dests.shape

(30, 5)